Following [this](https://pandera.readthedocs.io/en/stable/schema_inference.html)

### Schema Inference

*New in version 0.4.0*

With simple use cases, writing a schema definition manually is pretty straight-forward with pandera. However, it can get tedious to do this with dataframes that have many columns of various data types.

To help you handle these cases, the `infer_schema()` function enables you to quickly infer a draft schema from a pandas dataframe or series. Below is a simple example:

In [1]:
import pandas as pd
import pandera as pa

from pandera import Check, Column, DataFrameSchema

df = pd.DataFrame(
    {
        "column1": [5, 10, 20],
        "column2": ["a", "b", "c"],
        "column3": pd.to_datetime(["2010", "2011", "2012"]),
    }
)
print(df)
print()

schema = pa.infer_schema(df)  # NOTE.
print(schema)

"""
 <Schema DataFrameSchema(
     columns={
         'column1': <Schema Column(name=column1, type=DataType(int64))>
         'column2': <Schema Column(name=column2, type=DataType(object))>
         'column3': <Schema Column(name=column3, type=DataType(datetime64[ns]))>
     },
     checks=[],
     coerce=True,
     dtype=None,
     index=<Schema Index(name=None, type=DataType(int64))>,
     strict=False
     name=None,
     ordered=False,
     unique_column_names=False
 )>
""";

   column1 column2    column3
0        5       a 2010-01-01
1       10       b 2011-01-01
2       20       c 2012-01-01

<Schema DataFrameSchema(
    columns={
        'column1': <Schema Column(name=column1, type=DataType(int64))>
        'column2': <Schema Column(name=column2, type=DataType(object))>
        'column3': <Schema Column(name=column3, type=DataType(datetime64[ns]))>
    },
    checks=[],
    coerce=True,
    dtype=None,
    index=<Schema Index(name=None, type=DataType(int64))>,
    strict=False
    name=None,
    ordered=False,
    unique_column_names=False
)>


> These inferred schemas are **rough drafts** that *shouldn’t be used for validation without modification*!

You can modify the inferred schema to obtain the schema definition that you’re satisfied with.

For `DataFrameSchema` objects, the following methods create modified copies of the schema:
* add_columns()
* remove_columns()
* update_column()

For `SeriesSchema` objects:
* `set_checks()`

The section below describes two workflows for persisting and modifying an inferred schema.

## Schema Persistence

The schema persistence feature requires a pandera installation with the `io` extension.

See the [installation](https://pandera.readthedocs.io/en/stable/index.html#installation) instructions for more details.

There are two ways of persisting schemas, inferred or otherwise.

### Write to a Python script

You can also write your schema to a python script with `to_script()`:

In [2]:
# supply a file-like object, Path, or str to write to a file. If not
# specified, to_script will output the code as a string.
schema_script = schema.to_script()
print(schema_script)

from pandas import Timestamp
from pandera import DataFrameSchema, Column, Check, Index, MultiIndex

schema = DataFrameSchema(
    columns={
        "column1": Column(
            dtype="int64",
            checks=[
                Check.greater_than_or_equal_to(min_value=5.0),
                Check.less_than_or_equal_to(max_value=20.0),
            ],
            nullable=False,
            unique=False,
            coerce=False,
            required=True,
            regex=False,
            description=None,
            title=None,
        ),
        "column2": Column(
            dtype="object",
            checks=None,
            nullable=False,
            unique=False,
            coerce=False,
            required=True,
            regex=False,
            description=None,
            title=None,
        ),
        "column3": Column(
            dtype="datetime64[ns]",
            checks=[
                Check.greater_than_or_equal_to(
                    min_value=Timestamp(

As a python script, you can iterate on an inferred schema and use it to validate data once you are satisfied with your schema definition.

Other options (see links):
* [Write to YAML](https://pandera.readthedocs.io/en/stable/schema_inference.html#write-to-yaml)
* [Write to JSON](https://pandera.readthedocs.io/en/stable/schema_inference.html#write-to-json)